In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip3 install pickle5
!pip install --upgrade pandas
!pip uninstall openpyxl
!pip install openpyxl

     |████████████████████████████████| 256 kB 15.4 MB/s 
     |████████████████████████████████| 11.3 MB 15.8 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.


Found existing installation: openpyxl 2.5.9
Uninstalling openpyxl-2.5.9:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/openpyxl-2.5.9.dist-info/*
    /usr/local/lib/python3.7/dist-packages/openpyxl/*
Proceed (y/n)? y
  Successfully uninstalled openpyxl-2.5.9
     |████████████████████████████████| 242 kB 15.9 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from sklearn.metrics import r2_score
import itertools

import datetime
from dateutil.relativedelta import relativedelta
import pickle5 as pickle

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/cakd3 (1)/data(2016)_2.csv',index_col=0)
df.reset_index(inplace=True)
df=df[['납기일자','prod_name','sold_quant']]
df.납기일자 = pd.to_datetime(df['납기일자'])
a = df.groupby('prod_name')['sold_quant'].sum().sort_values(ascending=False)
a.index

Index(['PEMA-HR1000', 'CSA5000', 'PEMA-HR1500', 'PEMA-PR1000', 'PEMA-580FX',
       'PEMA-SR2000', 'PEMA-SR3000F', 'PEMA-500FR', 'PEMA-PR2000',
       'PEMA-SN400', 'CSA4000', 'AE', 'PEMA-PCM2000', 'PEMA-SR2000A',
       'PEMA-PR10001', 'PEMA-PCM2000B', 'PEMA-CR1000', 'PR1000',
       'PEMA-PCR3000E', 'CSA5000F', 'PEMA-CSA5000', 'PEMA-HR1000F', 'SRE-200',
       'PEMA-SR5000F', 'PEMA-SP1000', 'PEMA-HR1500F', 'PEMA-HR1000S',
       'PEMA-HR1000OPC', 'PEMA-PCR3000N', 'SRE-110', 'PEMA-SPR'],
      dtype='object', name='prod_name')

### GRU

In [ ]:
#####################################모델 학습###############################

from datetime import datetime
import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm 
import math

df_prod = pd.DataFrame(df.prod_name.value_counts())
df_prod.reset_index(inplace=True)
df_prod.rename(columns={'index': 'PRODNAME' , 'prod_name':'COUNT'}, inplace=True)


df_pred_model_0 = pd.DataFrame()
for i in tqdm(range(len(a.index))):
  df1 = df[df['prod_name']==a.index[i]]
  df1 = pd.DataFrame(df1.groupby('납기일자')['sold_quant'].sum())
  df1.reset_index(inplace=True)


  # 빈날짜 인덱스 만들기
  start_date = df1.납기일자.min()
  end_date = df1.납기일자.max()

  currentdate = start_date
  date_index = []
  while currentdate <=(end_date):
    date_index.append(currentdate)
    currentdate=currentdate+datetime.timedelta(days=1)

  data_index = pd.DataFrame(date_index,columns=['납기일자'])
  data_index['납기일자'] = pd.to_datetime(data_index['납기일자'])

  # 날짜 인덱스와 데이터 merge
  df1 = pd.merge(data_index,df1,on='납기일자',how='left')
  df1.fillna(0,inplace=True)
  df1.set_index('납기일자',inplace=True)
  df_prod1 = df1.copy()
  
  
  # train set, test set 만들기

  std_date = datetime.datetime.strptime("2020-11-24", "%Y-%m-%d") # 훈련용 기준날짜 21/4/25 이라고 가정. 데모에서 PC 날짜 바꾸고 today() 사용해서 테스트 함

  testdate_start = std_date - relativedelta(months=6)
  testdate_end = std_date
  traindate_end = testdate_start - relativedelta(days=1)
  traindate_start = (traindate_end - relativedelta(months=24)) # train data 기간 2년
  
  train_data = df_prod1[traindate_start:traindate_end+relativedelta(days=1)]
  test_data = df_prod1[testdate_start:testdate_end+relativedelta(days=1)]  
  

  scaler = MinMaxScaler().fit(train_data)
  train_scaled = scaler.transform(train_data)
  test_scaled = scaler.transform(test_data)

  
  # input dataset 만들기
  def create_dataset (X, look_back = 1):
      Xs, ys = [], []
  
      for i in range(len(X)-look_back):
          v = X[i:i+look_back]
          Xs.append(v)
          ys.append(X[i+look_back])
  
      return np.array(Xs), np.array(ys)
  

  LOOK_BACK = 30

  # X, y 나누기
  X_train, y_train = create_dataset(train_scaled,LOOK_BACK)
  X_test, y_test = create_dataset(test_scaled,LOOK_BACK)
  
 
  # GRU model 만들기
  def create_gru(units):
      model = Sequential()
      # Input layer
      model.add(GRU (units = units, return_sequences = True, 
                      input_shape = [X_train.shape[1], X_train.shape[2]]))
      model.add(Dropout(0.2)) 
      # Hidden layer
      model.add(GRU(units = units)) 
      model.add(Dropout(0.2))
      model.add(Dense(units = 1)) 
      #Compile model
      model.compile(optimizer='adam',loss='mse')
      return model
  model_gru = create_gru(64)
  
  
  def fit_model(model):
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                              patience = 10)
    history = model.fit(X_train, y_train, epochs = 100,  
                        validation_split = 0.2,
                        batch_size = 16, shuffle = False, 
                        callbacks = [early_stop])
    return history
  
  history_gru = fit_model(model_gru)

  # 스케일링 했던 값 원래대로 되돌리기
  y_test = scaler.inverse_transform(y_test)
  y_train = scaler.inverse_transform(y_train)
  
  # loss plot
  def plot_loss (history, model_name):
    plt.figure(figsize = (10, 6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Train vs Validation Loss for ' + model_name)
    plt.ylabel('Loss')
    plt.xlabel('epoch')
    plt.legend(['Train loss', 'Validation loss'], loc='upper right')

  plot_loss (history_gru, 'GRU')
  
  # 예측
  def prediction(model):
      prediction = model.predict(X_test)
      prediction = scaler.inverse_transform(prediction)
      return prediction
  prediction_gru = np.ceil(prediction(model_gru))
  
  
  # 실제값 vs 예측값 plot
  def plot_future(prediction, model_name, y_test):
      plt.figure(figsize=(10, 6))
      range_future = len(prediction)
      plt.plot(np.arange(range_future), np.array(y_test), 
              label='Test data')
      plt.plot(np.arange(range_future), 
              np.array(prediction),label='Prediction')
      plt.title('Test data vs prediction for '+ model_name)
      plt.legend(loc='upper left')
      plt.xlabel('Time (day)')
      plt.ylabel('Daily SOLD_QUANT')
  
  plot_future(prediction_gru, 'GRU', y_test)
  
  # 평가
  def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    r2 = r2_score(predictions, actual)
    print(model_name + ':')
    print('Mean Absolute Error: {:.4f}'.format(mae))
    print('Root Mean Square Error: {:.4f}'.format(rmse))
    print('R2 Score: {:.4f}'.format(r2))
    print('\n')
  
  print('---------',a.index[i],'-----------')
  evaluate_prediction(prediction_gru, y_test, 'GRU')
  
  # 실제값과 예측값 데이터프레임으로 출력
  y_test = list(itertools.chain.from_iterable(y_test))
  prediction = list(itertools.chain.from_iterable(prediction_gru))
  result = pd.DataFrame()
  result['y_test'] = y_test
  result['prediction']= prediction
  pred_date = test_data[-(len(result)):].reset_index()
  order_quant = pd.concat([pred_date,result], axis=1)
  order_quant.drop('sold_quant',axis=1,inplace=True)
  order_quant['prod_name']=a.index[i]
  df_pred_model = df_pred_model_0.append(order_quant)
  df_pred_model_0 = df_pred_model

  df_pred_model

  # # 모델 저장
  # model_gru.save(f'/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/gru_model_save/{a.index[i]}_gru.h5')




  # ################## 미래 예측 ##################################

  testdate_start = std_date - relativedelta(months=7)
  testdate_end = std_date - relativedelta(days=1)

  test_data = df_prod1[testdate_start:testdate_end+relativedelta(days=1)]  
  history = df_prod1[testdate_end-relativedelta(months=24):testdate_end+relativedelta(days=1)]
  scaler = MinMaxScaler()
  test_scaled = scaler.fit_transform(test_data)

  # 인풋 데이터셋 만들기
  def create_dataset (X, look_back = 1):
      Xs = []

      for i in range(len(X)-look_back):
          v = X[i:i+look_back]
          Xs.append(v)
      return np.array(Xs)

  LOOK_BACK=30
  X = create_dataset(test_scaled,LOOK_BACK)

  def prediction(model):
      prediction = model.predict(X)
      prediction = scaler.inverse_transform(prediction)
      return prediction
  prediction_gru = np.ceil(prediction(model_gru))

  # plot history and future
  def plot_multi_step(history, prediction):
    
    plt.figure(figsize=(15, 6))
    
    range_history = len(history)
    range_future = list(range(range_history, range_history +
                        len(prediction)))
    plt.plot(np.arange(range_history), np.array(history), 
                label='History')
    plt.plot(range_future, np.array(prediction),
              label='Forecasted for GRU')
      
    plt.legend(loc='upper right')
    plt.xlabel('Time step (day)')
    plt.ylabel('Sold_Quant')
        
  plot_multi_step(history,  prediction_gru)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
###################### 저장한 모델 불러와서 order_quant 뽑기 ##################

with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/수주분석테이블.pkl', "rb") as fh:
  수주분석테이블 = pickle.load(fh)

df = 수주분석테이블
df.reset_index(inplace=True)
df=df[['납기일자','제품명','판매수량']]
df.납기일자 = pd.to_datetime(df['납기일자'])
a = df.groupby('제품명')['판매수량'].sum().sort_values(ascending=False)
a.index

df_prod = pd.DataFrame(df.제품명.value_counts())
df_prod.rename(columns={'Unnamed: 0': '제품명' , 'PRODNAME':'COUNT'}, inplace=True)

df_pred_model_0 = pd.DataFrame()
for i in range(len(a.index)):
  df1 = df[df['제품명']==a.index[i]]
  df1 = pd.DataFrame(df1.groupby('납기일자')['판매수량'].sum())
  df1.reset_index(inplace=True)

  # 빈날짜 채우기
  start_date = df1.납기일자.min()
  end_date = df1.납기일자.max()

  currentdate = start_date
  date_index = []
  while currentdate <=(end_date):
    date_index.append(currentdate)
    currentdate=currentdate+datetime.timedelta(days=1)

  data_index = pd.DataFrame(date_index,columns=['납기일자'])
  data_index['납기일자'] = pd.to_datetime(data_index['납기일자'])

  df1 = pd.merge(data_index,df1,on='납기일자',how='left')
  df1.fillna(0,inplace=True)
  df1.set_index('납기일자',inplace=True)
  df_prod1 = df1.copy()

  from tensorflow.python.keras.models import load_model
  model_gru = load_model(f'/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/gru_model_save/{a.index[i]}_gru.h5')

  std_date = datetime.datetime.strptime("2021-04-25", "%Y-%m-%d") # 훈련용 기준날짜 21/4/25 이라고 가정. 데모에서 PC 날짜 바꾸고 today() 사용해서 테스트 함
  testdate_start = std_date - relativedelta(months=7)
  testdate_end = std_date - relativedelta(days=1)

  test_data = df_prod1[testdate_start:testdate_end+relativedelta(days=1)]  

  scaler = MinMaxScaler()
  test_scaled = scaler.fit_transform(test_data)

  # 인풋 데이터셋 만들기
  def create_dataset (X, look_back = 1):
      Xs = []

      for i in range(len(X)-look_back):
          v = X[i:i+look_back]
          Xs.append(v)
      return np.array(Xs)

  LOOK_BACK=30
  X = create_dataset(test_scaled,LOOK_BACK)

  def prediction(model):
      prediction = model.predict(X)
      prediction = scaler.inverse_transform(prediction)
      return prediction
  prediction_gru = np.ceil(prediction(model_gru))

  result = pd.DataFrame()
  preds = list(itertools.chain.from_iterable(prediction_gru))
  result['예측중량']= preds
  
  start_date = std_date
  end_date = std_date + relativedelta(months=6)

  currentdate = start_date
  date_index = []
  while currentdate <=(end_date):
    date_index.append(currentdate)
    currentdate=currentdate+datetime.timedelta(days=1)

  data_index = pd.DataFrame(date_index,columns=['납기일자'])
  data_index['납기일자'] = pd.to_datetime(data_index['납기일자'])

  pred = pd.concat([data_index,result],axis=1)
  pred['제품명']=a.index[i]
  중장기수주예측 = df_pred_model_0.append(pred)
  df_pred_model_0 = 중장기수주예측
  중장기수주예측.fillna(0,inplace=True)

  def to_zero(x):
    if x<0:
      return 0
    else:
      return x

  중장기수주예측['예측중량'] = 중장기수주예측['예측중량'].apply(to_zero)
  중장기수주예측['납기일자'] = pd.to_datetime(중장기수주예측['납기일자'])

중장기수주예측


,납기일자,예측중량,제품명
0,2021-04-25,84655.0,PEMA-HR1000
1,2021-04-26,45307.0,PEMA-HR1000
2,2021-04-27,0.0,PEMA-HR1000
3,2021-04-28,48884.0,PEMA-HR1000
4,2021-04-29,84230.0,PEMA-HR1000
...,...,...,...
179,2021-10-21,0.0,PEMA-SPR
180,2021-10-22,0.0,PEMA-SPR
181,2021-10-23,0.0,PEMA-SPR
182,2021-10-24,0.0,PEMA-SPR


In [ ]:
중장기수주예측.to_csv('/content/drive/MyDrive/5조_스마트팩토리/배송이/임시/gru_orderquant.csv')